In [ ]:
# !sudo pip install -e /workspace/src/aymurai

In [ ]:
from aymurai.spacy.display import DocRender
from aymurai.datasets.ar_juz_pcyf_10.docs import ArgentinaJuzgadoPCyF10DocsDataset
from aymurai.datasets.ar_juz_pcyf_10.annotations import (
    ArgentinaJuzgadoPCyF10LabelStudioAnnotations,
)

render = DocRender()
docs = ArgentinaJuzgadoPCyF10LabelStudioAnnotations(
    "/resources/annotations/label-studio/resos-annotations/30-nov/no-decision/"
).data
# docs = ArgentinaJuzgadoPCyF10DocsDataset().data
sample = docs[:10]

print(len(docs))

## Pipeline

In [ ]:
from aymurai.pipeline import AymurAIPipeline
from aymurai.models.flair.core import FlairModel
from aymurai.models.flair.utils import FlairTextNormalize
from aymurai.transforms.entity_subcategories.regex import RegexSubcategorizer
from aymurai.transforms.entity_subcategories.usem import USEMSubcategorizer
from aymurai.transforms.entity_subcategories.article import ArticleSubcategorizer


config = {
    "preprocess": [
        (FlairTextNormalize, {}),
    ],
    "models": [
        (
            FlairModel,
            {
                "basepath": "/resources/pipelines/examples/flair-simple/FlairModel",
                "split_doc": True,
                "device": "cpu",
            },
        )
    ],
    "postprocess": [
        (RegexSubcategorizer, {}),
        (
            USEMSubcategorizer,
                {
                    "category": "CONDUCTA",
                    # "subcategories": conduct_categories,
                    "subcategories_path": '/resources/pipelines/examples/flair-simple/UsemEmbeddings/conduct_options.txt',
                    "response_embeddings_path": "/resources/pipelines/examples/flair-simple/UsemEmbeddings/conduct_embeddings.npy",
                }
        ),
        (
            USEMSubcategorizer,
                {
                    "category": "CONDUCTA_DESCRIPCION",
                    # "subcategories": conduct_descr_categories,
                    "subcategories_path": '/resources/pipelines/examples/flair-simple/UsemEmbeddings/conduct_descr_options.txt',
                    "response_embeddings_path": "/resources/pipelines/examples/flair-simple/UsemEmbeddings/conduct_descr_embeddings.npy",
                }
        ),
        (
            USEMSubcategorizer,
                {
                    "category": "DETALLE",
                    # "subcategories": detail_categories,
                    "subcategories_path": '/resources/pipelines/examples/flair-simple/UsemEmbeddings/detail_options.txt',
                    "response_embeddings_path": "/resources/pipelines/examples/flair-simple/UsemEmbeddings/detail_embeddings.npy",
                }
        ),
        (
            USEMSubcategorizer,
                {
                    "category": "OBJETO_DE_LA_RESOLUCION",
                    "subcategories_path": '/resources/pipelines/examples/flair-simple/UsemEmbeddings/object_options.txt',
                    # "subcategories": object_categories,
                    "response_embeddings_path": "/resources/pipelines/examples/flair-simple/UsemEmbeddings/object_embeddings.npy",
                }
        ),
        (ArticleSubcategorizer, {})
    ],
    # "multiprocessing": {},
    "use_cache": False,
    # 'log_level': 'debug'
}

In [ ]:
pipeline = AymurAIPipeline(config)
results = pipeline.preprocess(sample)
results = pipeline.predict(results)
results = pipeline.postprocess(results)

In [ ]:
idx = -11

In [ ]:
render(results[idx])

In [ ]:
arts = [_ for _ in results[idx]["predictions"]["entities"] if _["label"] == "ART_INFRINGIDO"]
print(set([art["text"] for art in arts]))
arts

In [ ]:
conducts = [_ for _ in results[idx]["predictions"]["entities"] if _["label"] == "CONDUCTA"]
conducts